In [1]:
from datetime import datetime as dt
import psycopg2
import sys
import pandas as pd
import numpy as np

In [2]:
def get_filename_list(my_dir):
    import os
    a=[]
    for root, dirs, files in os.walk(my_dir):
        for name in files:
            if name.endswith((".txt")):
                new_name=os.path.join(root,name)
                #new_name=dirs+name
                a.append(new_name)
    return a

In [3]:
def get_address(ESIID):
    import psycopg2
    import sys
    import pandas as pd
    import numpy as np

    conn_string="host='localhost' dbname='BBS' user='postgres' password='Azurarai1994'"
    conn=psycopg2.connect(conn_string)
    cursor=conn.cursor()
    #Run SQL query
    command=""" SELECT *
    FROM public."CustomerRawData"
    WHERE "PremiseID"= '%s' """ %(ESIID)

    cursor.execute(command)
    conn.commit()

    #get record

    rec_consumption=cursor.fetchall()
    colnames = [desc[0] for desc in cursor.description]

    # convert to dataframe

    df1=pd.DataFrame(rec_consumption,columns=colnames)
    my_list=[df1.ServiceAddress[0],  df1.ServiceCity[0]]
    #my_address=' '.join(my_list)
    my_address=df1.ServiceAddress[0]
    city=df1.ServiceCity[0]
    
    return my_address,city



In [4]:
def get_location(address,city):
    from geopy.geocoders import GoogleV3
    citylist=["%s,",city]
    citystring=' '.join(citylist)
    geolocator = GoogleV3(format_string=citystring)
    address, (latitude, longitude) = geolocator.geocode(address)
    #print(address, latitude, longitude)
    return latitude,longitude

In [1]:
def get_weather(lat,lng,date):
    from darksky import forecast

    api_key = "83d639036099d4e5b4511f2df73f8ffb"
    PLACE = api_key, lat, lng
    my_forecast = forecast(*PLACE, time = date)
    return my_forecast


In [6]:
def weather_data(data,my_address,city):
    High=[]
    Low=[]
    humid=[]
    rise_T=[]
    set_T=[]

    [lat,lng]=get_location(my_address,city)

    for i in range(0,len(data['StartTime'])):
        my_date=dt.strptime(data.StartTime[i], '%Y-%m-%d %H:%M:%S')
        my_date=my_date.isoformat()
        my_forecast = get_weather(lat,lng,my_date)

        High.append(my_forecast['daily']['data'][0]['temperatureHigh'])
        Low.append(my_forecast['daily']['data'][0]['temperatureLow'])
        humid.append(my_forecast['daily']['data'][0]['humidity'])
        rise_T.append(my_forecast['daily']['data'][0]['sunriseTime'])
        set_T.append(my_forecast['daily']['data'][0]['sunsetTime'])


    #if("cloudCover" %in% colnames(fore_cast.daily)){cloudy[i]=fore_cast.daily.cloudCover}else{cloudy[i]='NA'}
    data['High']= High
    data['Low']= Low
    data['humid']=humid
    data['rise_T']=rise_T
    data['set_T']=set_T
    
    return data

In [7]:
def get_weather_data(filename):
    import pandas as pd
    data=pd.read_csv(filename)
    ESIID=filename[57:-4]
    my_address,city=get_address(ESIID)
    new_data=weather_data(data,my_address,city)
    fs=['C:/Users/ruchi/Desktop/WeatherData/weather_for_',ESIID,'.csv']
    file_save=''.join(fs)
    new_data.to_dense().to_csv(file_save, index = False, sep=',', encoding='utf-8') 
    return new_data

In [ ]:
my_dir = "C:/Users/ruchi/Desktop/Result1/C100_RES/"
fnames = get_filename_list(my_dir)
for i in range(91,101):
    f=fnames[i]
    wthr_data= get_weather_data(f)
    